# Building custom layers in tensorflow

In [1]:
import tensorflow as tf

When we enable eager execution, all operations execute immediately and return their values to Python without needing a Session.run(). 

In [3]:
tf.enable_eager_execution()

In keras layers are objects, so to construct a layer we just need to construct an object. Most layers take the first argument as the number of output dimensions or channels.

In [11]:
layer = tf.keras.layers.Dense(100)

Providing the input dimension is often unnecessary, because it can be inferred the first time the layer is used. But we can provide it manually which is helpful in complex models.

In [12]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

Using a layer

In [13]:
layer(tf.zeros([10,5]))

<tf.Tensor: id=70, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

## Implementing custom layers

The best way to do so is by extending the tf.keras.layer class and implementing:
- '\_\_init_\_\', for initializing the layer with inputs and stuff
- build, here we know the shapes of the input tensors and can do the rest of the initialization
- call, here we do the forward computation

In [28]:
# Sub class of tf.keras.layers.Layer
class CustomLayer(tf.keras.layers.Layer):
    # initializing with input and output
    def __init__(self, outputs):
        super(CustomLayer, self).__init__()
        self.num_outputs = outputs
    
    # initializing the kernel or our variables
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel", shape = [int(input_shape[-1]), self.num_outputs])
    
    # forward computation (with our own variables or otherwise)
    def call(self, input):
        return tf.matmul(input, self.kernel)

In [31]:
layer = CustomLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'custom_layer_5/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.51974714, -0.15228558, -0.53701925,  0.44154543,  0.51965314,
         0.6142784 ,  0.57382613,  0.45008498,  0.42581087,  0.4227665 ],
       [-0.5652827 ,  0.1948865 ,  0.5958591 ,  0.15037733,  0.56816083,
         0.2699899 ,  0.24857426,  0.5635964 ,  0.4462976 ,  0.51113707],
       [-0.23664653,  0.39654619,  0.4059072 ,  0.550376  ,  0.05814493,
        -0.38133746, -0.39821196, -0.43864262,  0.44912165, -0.49076396],
       [ 0.00203037, -0.30667496, -0.4415112 , -0.45286384, -0.47903994,
        -0.06180227,  0.4898668 ,  0